# Does time == money?
## A deep dive into the time spent by kagglers in responding to the 2019 kaggle survey.
![](https://www.incimages.com/uploaded_files/image/970x450/getty_649699796_348644.jpg)

It's October 8th, 2019, you check your email inbox. "Oh look" you think to yourself, "Kaggle's 2019 survey is out!". Of course you intend on taking the survey, but depending on who you are two different thoughts might be going through your head:

- "Lets take some time grab a coffee and get to answering some questions"
- "I don't have any time! Lets get this over quick."

In this analysis we will look to gain insights from the 2019 Kaggle Survey by focusing on the time respondents took to take ths survey. We will see if preferences differ between *fast* responders compared to *slow* responders.

Specifically we will dive into 5 key findings when looking into responders which I will expand upon below.

You may be asking yourself, "Why take a look at the time taken by respondents?" One could argue that the time a respondent takes on a survey could signal many things about their enagement with the platform. Those taking longer could be thinking through the questions in more detail, and may value the survey's importance more. You could also argue that those who take less time responding, while interested in kaggle, are not devoted enough to take a survey- or simply too busy- and are prime candidates to try to draw closer into the community. Looking at response time may also lead to revelaing bias in the survey results. Could answers be less valuable if they were clicked by someone who didn't really take the time to read the question?

So sit back, take your time and enjoy the analysis!

In [ ]:
import numpy as np # for mathing things
import pandas as pd # pandas are my favorite animal
import matplotlib.pylab as plt # pylab time
import seaborn as sns # sns stands for Samuel Norman "Sam" Seaborn, a fictional character portrayed by Rob Lowe on the television serial drama The West Wing.
from sklearn.preprocessing import KBinsDiscretizer
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

plt.style.use('seaborn')
sns.set(style="whitegrid")
sns.set_color_codes("pastel")
color_pal = [x['color'] for x in plt.rcParams['axes.prop_cycle']]
pd.set_option('max_columns', 100) # So we can see more columns

In [ ]:
# Read input
qs = pd.read_csv('../input/kaggle-survey-2019/questions_only.csv')
mcr = pd.read_csv('../input/kaggle-survey-2019/multiple_choice_responses.csv', low_memory=False)
mcr = mcr.drop(0) # Drop col
mcr['Time_Taken'] = pd.to_numeric(mcr['Time from Start to Finish (seconds)']) # Make numeric
mcr['count'] = 1 # For making counts
mcr['Time_Taken_minutes'] = mcr['Time_Taken'] / 60

# How long, on average, did respondents take?
Just for some perspective of amlost 20,000 respondents.
- The median response time was 540 seconds (9 minutes!)
- The fastest respondent look only 23 seconds.
- The longest respondent took 843612 seconds (9.7 days!) - how is this possible?

In [ ]:
print('Median time taken: {:0.0f} minutes'.format(mcr['Time_Taken_minutes'].median()))
print('Maximum time taken: {:0.0f} minutes'.format(mcr['Time_Taken_minutes'].max()))
print('Minimum time taken: {:0.2f} minutes'.format(mcr['Time_Taken_minutes'].min()))

## Assumptions:
- Right off the bat we can see the response time may be skewed by bad data or people who finished the survey in more than one sitting.
- Lets assume that any respondant that look longer than 60 minutes to respond is not legitimate and ignore them for the remainder of this analysis. Lets also assume anyone who completed the survey in under 1 minute did not invest the approriate amount to make meaningful responses to the questions.
- This drops roughly 2000 responses from our dataset.

In [ ]:
mcr_small = mcr.query('Time_Taken_minutes <= 60 and Time_Taken_minutes >= 1').copy()
print(f'Number of respondents before removing outliers {len(mcr)}, and after removing {len(mcr_small)}')

Looking at the distribution of our dataset after removing outliers, we can see that there is a somewhat normal distribution around 7 minutes, with a long tail up to 60 minutes. The distribution appears to rise to the left of the mean with what I like to call the **superfast** responders.

In [ ]:
fig, ax = plt.subplots()
mcr_small['Time_Taken_minutes'].plot(kind='hist',
                                     bins=60,
                                     figsize=(15, 5),
                                     color=color_pal[1])
fig.suptitle('Distribution of minutes taken to respond*', fontsize=20)
plt.text(30, -200, '*Including responses greater than 1 minute and less than 1 hour')
plt.show()

# Finding 1: Kagglers combined for over 131 days in taking the survey.
With the assumptions above, we can now compute that the kaggle community, combined spent over 130 days responding to the survey!

In [ ]:
tot_days = mcr_small['Time_Taken_minutes'].sum() / 60 / 24
print(f'The kaggle community combined response time is approximately {tot_days:0.0f} days')


Lets take it a step further. Since we have the respondents approximate annual salary we can compute the total financial value of the time taken to respond to the survey. Each respondent selected a time bracket for their annual salary earnings. To simplify we will assume the respondent earns the average ammount for their bracket. We will assume that those in the `> $500,000` bracket spot average a \$750,000 salary. Lets also assume that each person works 52 weeks, 40 hours a week. Using these assumptions we can calculate a value assoicated with each respondents time per minute. Finally, we compute that the combined value of responses was roughly $57,840- if kaggle were to pay respondents their salary for the time they spent this is how much they would've had to pay.

## We can extrapolate from salary data that the value of time spent by survey responders is over \$57,000! 

In [ ]:
ann_inc_dict = {}
for inc in mcr_small['Q10'].unique():
    try:
        inc_split = inc.replace('$','').replace(',','').split('-')
        inc_aprox_group = (int(inc_split[1]) + int(inc_split[0])) / 2
        ann_inc_dict[inc] = inc_aprox_group
    except:
        if inc == '> $500,000':
            ann_inc_dict[inc] = 750000
        else:
            pass # Ignore nan responses
mcr_small['Annual_Income_Aprox'] = mcr_small['Q10'].map(ann_inc_dict)
# Assume 52 weeks in a year worked and 40 hours a week
mcr_small['Aprox_inc_min'] = mcr_small['Annual_Income_Aprox'] / 52 / 40 / 60
mcr_small['Aprox_cost_of_response'] = mcr_small['Aprox_inc_min'] * mcr_small['Time_Taken_minutes']
tot_time_cost = mcr_small['Aprox_cost_of_response'].sum()
print(f'With some assumptions, the total cost of the survey respondents time taken is roughly ${tot_time_cost:0.2f}')

Below we see a distribution of the cost of response time for each respondent, in a log scale with 10<sup>-2</sup> costing \$0.01 and 10<sup>2</sup> costing over \$200. Interestingly we see two "humps" in the distribution, for the low salary responses and the higher salary responses.

In [ ]:
# https://www.kaggle.com/robikscube/5-findings-from-looking-at-survey-response-speed/edit
# histogram on linear scale
fig, ax = plt.subplots(figsize=(15, 10))
hist, bins, _ = plt.hist(mcr_small['Aprox_cost_of_response'], bins=50)

# histogram on log scale. 
# Use non-equal bin sizes, such that they look equal on log scale.
logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
plt.subplot(212)
plt.hist(mcr_small['Aprox_cost_of_response'], bins=logbins)
plt.suptitle('Distribution of the Cost of Respondents Time', fontsize=20, y=0.5)
plt.xlabel('$ cost of time taken to respond')
plt.xscale('log')
plt.show()

**Fun fact**: The most expensive single person responding to the survey is someone who makes over \$500,000 a year and spent over 45 minutes responding, costing time over $278 of their time!

In [ ]:
print('Max cost of response ${:0.2f}'.format(mcr_small['Aprox_cost_of_response'].max()))

# Finding 2: Younger kagglers took the least amount of time, and middle agers took the most!

If we look at the median time taken by respondents based on their age bracket we can clearly see that younger responders took less time. There are several possible explainations for this finding. Younger responders may have less experience, so they would not need to think thoroughly about their response. Younger responders could also just be less patient for taking long surveys. We also see that time spent on respondes does not continue to increase after the 40-44 age bracket, but instead shows a slight decrease.

In [ ]:
fig, ax = plt.subplots()

mcr.groupby('Q1') \
    .median()['Time_Taken_minutes'] \
    .plot(kind='bar',
          figsize=(15, 5),
          color=color_pal[2])
fig.suptitle('Survey Response Time by Age', fontsize=20)
plt.xlabel('')
plt.ylabel('Minutes to Respond')
plt.show()

Similar insights can be found when looking at the survey response by education of the responder. Masters Degree responders have much more long response times compared to those with less education.

In [ ]:
mcr_small['Q4_short'] = mcr_small['Q4'].replace({'No formal education past high school' : 'No college',
            'Some college/university study without earning a bachelor’s degree' : 'Some college'})

q4_order = ['No college',
            'Some college',
            'Professional degree',
            'Bachelor’s degree',
            'Master’s degree',
            'Doctoral degree',
            'I prefer not to answer',
           ]
fig, ax= plt.subplots(figsize=(15, 5))
sns.stripplot(x='Q4_short', y='Time_Taken_minutes', data=mcr_small, order=q4_order, ax=ax)#, alpha=0.1)
fig.suptitle('Survey Response Time by Education', fontsize=20)
plt.xlabel('')
plt.ylabel('Minutes to Respond')
plt.show()

# Finding 3: Nigerian kagglers took their time answering; Australians are speedy.
Next we will look at kaggers by their country of orgin. We will look at the *median* value to minimize the impact of outliers. We also filter down to countries with at least 200 respondents to focus on countries where we have a decent sample size. As you can see Nigerian kagglers took over a minute longer to respond (11.07 minutes) to the survey than the next country, Japan (9.72 minutes). It also looks like native english speaking countries (Australia, USA, UK, and Canada) were able to respond quite quickly. Kaggle may want to think about translating the survey for non-native speakers in future iterations.

In [ ]:
fig, ax = plt.subplots()

country_at_least_200resp = mcr.groupby('Q3').count()[['count']].query('count > 200').index.values

mcr_small['Q3_abbrv'] = mcr_small['Q3'].str.replace('United Kingdom of Great Britain and Northern Ireland','UK/GB/NorIr') \
                                       .str.replace('United States of America', 'USA')

ax = mcr_small.query('Q3 in @country_at_least_200resp') \
    .groupby('Q3_abbrv')['Time_Taken_minutes'] \
    .median() \
    .sort_values() \
    .plot(kind='barh',
          figsize=(12, 10),
         color=color_pal[0])
x_offset = -0.6
y_offset = -0.37
for p in ax.patches:
    b = p.get_bbox()
    val = "{:0.2f}".format(b.x1 + b.x0)        
    ax.annotate(val, (b.x1 + x_offset, b.y1 + y_offset), fontsize=10, color='w')
plt.ylabel('Which Country do You Current Reside?')
plt.xlabel('Minutes to Respond')
fig.suptitle('Median Response Time by Country', fontsize=20, y=0.92)
plt.show()

# Finding 4: In school or don't have a job? Then you probably aren't taking a long time to respond.

Suprisingly students and un-employed kagglers, who you might think have the most free time, were some of the quickest to respond to the survey. Project Managers and Database Engineers however were taking their sweet time. This could be because Students have less experience, and therefore less to think about when responding to the survey.

In [ ]:
ax = mcr_small.groupby('Q5').median()['Time_Taken_minutes'] \
    .sort_values() \
    .plot(kind='barh',
          figsize=(15, 6),
          color=color_pal[3],
          title='Time Taken by Job Type')
x_offset = -0.4
y_offset = -0.4
for p in ax.patches:
    b = p.get_bbox()
    val = "{:0.2f}".format(b.x1 + b.x0)        
    ax.annotate(val, (b.x1 + x_offset, b.y1 + y_offset), fontsize=10, color='white')
plt.show()

# Finding 5: Comparing super fast responders vs ultra slow responders

Lastly we will group the responders into four different groups based on their response times. We will compare the responses the two extreme groups, the super fast and ultra slow responding groups. Now we can look at the division of ultra fast responders compared to super slow responders for specific answers to survey questions.

In [ ]:
kbd= KBinsDiscretizer(4, encode='ordinal')
mcr_small['Time_Taken_bin'] = kbd.fit_transform(mcr_small[['Time_Taken']])
print(f'Super fast group: {kbd.bin_edges_[0][0]:.0f} seconds to {kbd.bin_edges_[0][1]:.0f} seconds')
print(f'Faster than average group: {kbd.bin_edges_[0][1]:.0f} seconds to {kbd.bin_edges_[0][2]:.0f} seconds')
print(f'Slower than average group: {kbd.bin_edges_[0][2]:.0f} seconds to {kbd.bin_edges_[0][3]:.0f} seconds')
print(f'Ultra slow group: {kbd.bin_edges_[0][3]:.0f} seconds to {kbd.bin_edges_[0][4]:.0f} seconds')

mcr_small['Time_Taken_bin_name'] = mcr_small['Time_Taken_bin'].map({0:'Super fast',
                                 1:'Faster than average',
                                 2:'Slower than average',
                                 3:'Ultra slow'})

Super fast responders have less experience coding.

In [ ]:
mcr_small.query('Time_Taken_bin in (0, 3)').groupby(['Time_Taken_bin_name','Q15']) \
    .count() \
    .unstack('Time_Taken_bin_name')['count'] \
    .T.apply(lambda x: x / x.sum()) \
    [['I have never written code', '< 1 years', '1-2 years',
       '3-5 years', '5-10 years', '10-20 years','20+ years']].T \
    .plot(kind='barh',
          figsize=(15, 5),
          stacked=True,
          title=qs['Q15'].values[0])
plt.show()

Fast responders are more likely to use spreadsheets. Slow responders are ones that use cloud software.

In [ ]:
mcr_small.query('Time_Taken_bin in (0, 3)').groupby(['Time_Taken_bin_name','Q14']) \
    .count() \
    .unstack('Time_Taken_bin_name')['count'] \
    .T.apply(lambda x: x / x.sum()).T \
    .sort_values('Super fast') \
    .plot(kind='barh', figsize=(15, 5), stacked=True)
plt.show()

Responders who took their time responding have recently started using ML Methods.

In [ ]:
mcr_small.query('Time_Taken_bin in (0, 3)').groupby(['Time_Taken_bin_name','Q8']) \
    .count() \
    .unstack('Time_Taken_bin_name')['count'] \
    .T.apply(lambda x: x / x.sum()).T \
    .sort_values('Super fast') \
    .plot(kind='barh',
          figsize=(15, 5),
          stacked=True,
          title=qs['Q7'].values[0])
plt.show()

Slow responders go against the grain when suggesting which programming language aspiring data scientists should learn.

In [ ]:
mcr_small.query('Time_Taken_bin in (0, 3)').groupby(['Time_Taken_bin_name','Q19']) \
    .count().query('count > 50')  \
    .unstack('Time_Taken_bin_name')['count']\
    .T.apply(lambda x: x / x.sum()).T \
    .sort_values('Super fast') \
    .plot(kind='barh', figsize=(15, 5), stacked=True,
          title=qs['Q19'].values[0])
plt.show()

# Takeaways

If you're reading this then you've taken valuable time out of your day to read my analysis, and I really apprecaite that! I hope that this different angle on evaluating the 2019 kaggle survey responses was fresh and insightful. Some suggestions that kaggle might glean from this analysis that might be helpful when designing next years survey are:
- Consider providing translated versions of the survey for non-native English speaking kagglers. Since kaggle is a diverse community with members from all around the world, we find that english speakers clearly can respond to the survey faster than non english speaking responders.
- Add some focused questions for students. Even though they make up a large portion (~20%) of the respondents, they spent considerably less time responding to the survey compared to non-students. Instead of only asking questions about what software and technologies respondents have already used, kaggle could ask more questions about what technologies kagglers hope to learn or use in the future. This would capture some more interesting information specifically from the student population.
- If possible, in future surveys it would be fun to see the response time break down per question. Using this data would allow for even deeper analysis into which groups of people engage with a specific question types.

Thanks for reading!